In [ ]:
import numpy as np
import pandas as pd

DATA_FILE = "../data/LOG002.BIN"

# dtype matches the packed C struct of three 32-bit floats (little-endian)
RMC_DTYPE = np.dtype([('lat', '<f4'), ('lon', '<f4'), ('vel', '<f4')])

data = np.fromfile(DATA_FILE, dtype=RMC_DTYPE)


pose_df = pd.DataFrame(data)

print(pose_df.head())



In [ ]:
# Convert collected pose_data into a DataFrame and plot relative trajectory
import plotly.express as px
import math


# Reference (start) point
lat0 = pose_df.loc[0, 'lat']
lon0 = pose_df.loc[0, 'lon']

# Approximate conversion: meters per degree latitude ~= 111132, longitude scaled by cos(lat)
meters_per_deg_lat = 111132.0
meters_per_deg_lon = 111320.0 * math.cos(math.radians(lat0))

pose_df['x_m'] = (pose_df['lon'] - lon0) * meters_per_deg_lon
pose_df['y_m'] = (pose_df['lat'] - lat0) * meters_per_deg_lat

# Create a simple scatter plot of trajectory (x east, y north)
fig = px.scatter(pose_df, x='x_m', y='y_m', color='vel',
                    title='Trajectory relative to start (meters)',
                    labels={'x_m': 'East (m)', 'y_m': 'North (m)', 'vel': 'Speed (knots)'} ,
                    hover_data=['lat', 'lon', 'vel'])
fig.update_traces(mode='lines+markers')
fig.update_layout(yaxis=dict(scaleanchor='x', scaleratio=1))
fig.show()

In [ ]:
# Convert collected pose_data into a DataFrame and plot relative trajectory
import plotly.express as px


pose_df['vel_mean'] = pose_df['vel'].rolling(window=5, min_periods=1, center=False).mean()




# Create a simple scatter plot of trajectory (x east, y north)
fig = px.scatter(x=range(len(pose_df)), y=pose_df['vel_mean'])
fig.update_traces(mode='lines+markers')
fig.show()

In [ ]:
import plotly.express as px


center_lat = float(pose_df['lat'].mean())
center_lon = float(pose_df['lon'].mean())

# fig = px.scatter_map(
#     pose_df,
#     lat='lat',
#     lon='lon',
#     color='vel',
#     zoom=17,
#     center={"lat": center_lat, "lon": center_lon},
#     hover_data=['lat', 'lon', 'vel'],
#     title="GPS track (lat/lon)"
# )

fig = px.line_map(
    pose_df,
    lat='lat',
    lon='lon',
    zoom=15,
    center={"lat": center_lat, "lon": center_lon},
    hover_data=['lat', 'lon', 'vel'],
    title="GPS track (lat/lon) - connected"
)

fig.show()